In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatterExponent
from DTA.enrichment_plotters import polar_plot, get_file_list, get_helices, read_rep
from DTA.polarDensity_helper import Coord_Get, get_header_info
from DTA.site_distributions import outline_site, make_symmetric_sites, combine_sites, plot_density, Site
plt.rcParams['axes.grid'] = False 

In [ ]:
# Enrichment plot colormap
from matplotlib.colors import ListedColormap
depleted = plt.cm.get_cmap('RdGy_r', 256)
enriched = plt.cm.get_cmap('bwr', 256)
newcolors = np.concatenate([depleted(np.linspace(0.35, 0.5, 128)), enriched(np.linspace(0.5,1,128))])
my_cmap = ListedColormap(newcolors)

# Read and plot the distribution
In VMD:
1. load the trajectory of the empty membrane
2. modify do_get_counts.tcl to have the total area of the site
3. source do_get_counts.tcl
4. It will save to a file called counts_[area].out
5. Provide the path here:

In [ ]:
area = 96
root = Path(f"../accessible_area/")
bulk_counts_path = root.joinpath(f"POPG_counts_{area}.dat")
bulk_counts = np.loadtxt(bulk_counts_path)

frequencies = np.bincount(bulk_counts.astype(int).flatten())
probabilities = frequencies/np.sum(frequencies)
fig, ax = plt.subplots()
ax.plot(range(len(probabilities)), probabilities)
ax.set_ylabel("Probability")
ax.set_xlabel(f"Number of beads in an area about {area} "+r"$\AA^2$")
bulk_mode = np.argmax(probabilities)
ax.vlines([bulk_mode], 0, np.max(probabilities), color = 'black', linestyles='dashed', label = f"mode={bulk_mode}")
ax.legend()

In [ ]:
probabilities[0]

In [ ]:

fig, axes = plt.subplots(2, 1, figsize=(3,6), sharex=True, sharey=True)

area = 47.5
root = Path(f"/home/ems363/Projects/MIE_paper/ELIC_Data/npeak_data/")
#root = Path(f"/Users/ezry/Projects/Polar_Binning_DeltaG/accessible_area/")
bulk_counts_path = root.joinpath(f"POPG_counts_{area}.out")
bulk_counts = np.loadtxt(bulk_counts_path)
frequencies = np.bincount(bulk_counts.astype(int).flatten())
probabilities_cheng = frequencies/np.sum(frequencies)
axes[0] = plot_bulk_counts(bulk_counts, axes[0])
axes[0].set_xlabel(None)
axes[0].set_ylabel("P(n) Site 1")


area = 81
bulk_counts_path = root.joinpath(f"POPG_counts_{area}.out")
bulk_counts = np.loadtxt(bulk_counts_path)
frequencies = np.bincount(bulk_counts.astype(int).flatten())
probabilities_ulens = frequencies/np.sum(frequencies)
axes[1] = plot_bulk_counts(bulk_counts, axes[1])
axes[1].set_xlabel("n")
axes[1].set_ylabel("P(n) Site 2")
# heights, edges = np.histogram(np.mean(bulk_counts, axis=1))
# centers = edges[:-1] + np.average(edges[0:1]) - edges[0]
# heights = heights/np.sum(heights)
# axes[1].plot(centers, heights)

fig.subplots_adjust(left=0.2)
plt.savefig(f"POPG_bulk_distribution_combo.pdf")

In [ ]:
np.sum(probabilities_ulens[12:])

In [ ]:
np.sum(probabilities_cheng[12:])

In [ ]:
area = 112
root = Path(f"/home/ems363/Projects/Polar_Binning_DeltaG/accessible_area/")
bulk_counts_path = root.joinpath(f"POPE_counts_{area}.out")
print(bulk_counts_path)

bulk_counts = np.loadtxt(bulk_counts_path)

fig, ax = plt.subplots()
ax = plot_bulk_counts(bulk_counts, ax)
plt.savefig("POPE_bulk_distribution_50frms.pdf")